In [ ]:
!pip install import-ipynb

import tensorflow as tf #모듈 
import numpy as np
import os

In [387]:
def split_xinput_target(chunk): 
    input_text = chunk[:-1]
    return input_text
def split_yinput_target(chunk): 
    output_text = [int(chunk[-1])]
    return output_text

In [384]:
player_name = 'mrugesh'
txt_file = open("database/"+ player_name +".txt")
train_data = [i for i in txt_file.readline()]

In [385]:
char2idx = {'R':0,'P':1,'S':2}
idx2char = {0:'R',1:'P',2:'S'}

def text_to_int(text):
    return np.array([char2idx[c] for c in text])
def int_to_text(integer):
    return idx2char[integer]

# train_data = np.expand_dims(train_data, axis=1)[0]
train_data = text_to_int(train_data)
# print(train_data)

In [386]:
seq_length = 3
examples_per_epoch = len(train_data)//(seq_length+1)

sequences = train_data[:examples_per_epoch*seq_length].reshape(examples_per_epoch, seq_length)

# sequences = train_data.batch(seq_length+1, drop_remainder=True)
x_dataset = np.apply_along_axis(split_xinput_target, 1, sequences)
x_dataset = np.reshape(x_dataset, (1, np.shape(x_dataset)[0], np.shape(x_dataset)[1]))
y_dataset = np.apply_along_axis(split_yinput_target, 1, sequences)
y_dataset = np.reshape(y_dataset, (1, np.shape(y_dataset)[0], np.shape(y_dataset)[1]))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [370]:
# print(np.shape(x_dataset))
# print(x_dataset[0])
# print(np.shape(y_dataset))
# print(y_dataset[0])

(1, 749, 2)
[[0 1]
 [2 0]
 [1 1]
 ...
 [2 0]
 [1 1]
 [0 0]]
(1, 749, 1)
[[1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1

In [380]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.Input(batch_input_shape=(1,None,2)),#np.shape(x_dataset)),
        # tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                  #batch_input_shape=[2]), 
        tf.keras.layers.LSTM(units=rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.7),	
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    return model

In [381]:
BATCH_SIZE = examples_per_epoch
VOCAB_SIZE = 3
EMBEDDING_DIM = 8
RNN_UNITS = 1024

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
#model.build(tf.TensorShape([1]))
model.summary() 

Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_76 (LSTM)              (1, None, 1024)           4206592   
                                                                 
 dropout_50 (Dropout)        (1, None, 1024)           0         
                                                                 
 dense_76 (Dense)            (1, None, 64)             65600     
                                                                 
 dense_77 (Dense)            (1, None, 3)              195       
                                                                 
Total params: 4,272,387
Trainable params: 4,272,387
Non-trainable params: 0
_________________________________________________________________


In [373]:
def loss(labels, logits): #logits = probability distribution
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [382]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [383]:
model.compile(optimizer="adam", 
            loss="sparse_categorical_crossentropy",
            metrics=['accuracy'])
#model.save_weights(checkpoint_path.format(epoch=0))

if os.path.exists(checkpoint_dir + "checkpoint"):
    history = model.fit(x_dataset, y_dataset, epochs=40, callbacks=[cp_callback])
else:
    history = model.load_weights(checkpoint_path)

Epoch 1/40
1/1 [==============================] - ETA: 0s - loss: 1.1095 - accuracy: 0.3151
Epoch 1: saving model to training_1\cp.ckpt
1/1 [==============================] - 12s 12s/step - loss: 1.1095 - accuracy: 0.3151
Epoch 2/40
1/1 [==============================] - ETA: 0s - loss: 1.1235 - accuracy: 0.4206
Epoch 2: saving model to training_1\cp.ckpt
1/1 [==============================] - 10s 10s/step - loss: 1.1235 - accuracy: 0.4206
Epoch 3/40
1/1 [==============================] - ETA: 0s - loss: 1.0527 - accuracy: 0.4219
Epoch 3: saving model to training_1\cp.ckpt
1/1 [==============================] - 9s 9s/step - loss: 1.0527 - accuracy: 0.4219
Epoch 4/40
1/1 [==============================] - ETA: 0s - loss: 1.0430 - accuracy: 0.5274
Epoch 4: saving model to training_1\cp.ckpt
1/1 [==============================] - 9s 9s/step - loss: 1.0430 - accuracy: 0.5274
Epoch 5/40
1/1 [==============================] - ETA: 0s - loss: 1.0331 - accuracy: 0.4753
Epoch 5: saving model to

In [378]:
def player(prev_play, opponent_history=["R","R"]):
    checkpoint_path = "training_1/cp.ckpt"

    if not prev_play == '':
        opponent_history.append(prev_play)
    guess_arr = text_to_int(opponent_history[-2:])
    guess_arr = np.reshape(guess_arr, (1,1,2))
    
#     BATCH_SIZE = examples_per_epoch
#     VOCAB_SIZE = 3
#     EMBEDDING_DIM = 8
#     RNN_UNITS = 1024

#     model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
#     model.compile(optimizer="adam", 
#             loss="sparse_categorical_crossentropy",
#             metrics=['accuracy'])
#     history = model.load_weights(checkpoint_path)
    guess = model.predict(guess_arr, verbose=0)
    #print(guess[0][0])
    
    return int_to_text(np.argmax(guess[0][0]))


In [379]:
player("P")

1/1 [==============================] - 0s 451ms/step


'R'